In [ ]:
import pandas as pd
import numpy as np

df = pd.read_csv('IMDB Dataset.csv', on_bad_lines='skip', engine='python')
print(df['sentiment'].value_counts())  # Cek distribusi label

sentiment
positive    25000
negative    25000
Name: count, dtype: int64


In [ ]:
from sklearn.model_selection import train_test_split
train_df, test_df = train_test_split(df, test_size=0.2, random_state=42)

In [ ]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Contoh tokenisasi satu teks:
encoded_input = tokenizer("Hello, world!", return_tensors='pt')
print(encoded_input)  # Lihat input_ids, attention_mask

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

{'input_ids': tensor([[ 101, 7592, 1010, 2088,  999,  102]]), 'token_type_ids': tensor([[0, 0, 0, 0, 0, 0]]), 'attention_mask': tensor([[1, 1, 1, 1, 1, 1]])}


In [ ]:
from torch.utils.data import Dataset
class IMDBDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['review']
        label = 1 if self.df.iloc[idx]['sentiment'] == 'positive' else 0
        encoded_text = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded_text['input_ids'].flatten(),
            'attention_mask': encoded_text['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
from transformers import BertForSequenceClassification
from torch.optim import AdamW  # Ambil AdamW dari torch.optim

model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=2  # Positif/Negatif
)


model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
from torch.utils.data import Dataset, DataLoader
import torch

class IMDBDataset(Dataset):
    def __init__(self, df, tokenizer, max_len=512):
        self.df = df
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.df)  # Kembalikan jumlah baris dalam DataFrame

    def __getitem__(self, idx):
        text = self.df.iloc[idx]['review']
        label = 1 if self.df.iloc[idx]['sentiment'] == 'positive' else 0
        encoded_text = self.tokenizer(
            text,
            max_length=self.max_len,
            padding='max_length',
            truncation=True,
            return_tensors='pt'
        )
        return {
            'input_ids': encoded_text['input_ids'].flatten(),
            'attention_mask': encoded_text['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }

In [ ]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = model.to(device)

In [ ]:
train_dataset = IMDBDataset(train_df, tokenizer)
train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True, num_workers=4, pin_memory=True)
test_dataset = IMDBDataset(test_df, tokenizer)
test_loader = DataLoader(test_dataset, batch_size=8, shuffle=False, num_workers=4, pin_memory=True)
print("Jumlah batch:", len(train_loader))  # Harus tidak error

Jumlah batch: 5000


/usr/local/lib/python3.11/dist-packages/torch/utils/data/dataloader.py:624: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(


In [ ]:
from torch.cuda.amp import autocast, GradScaler

# Inisialisasi optimizer
optimizer = AdamW(model.parameters(), lr=2e-5)

# Inisialisasi mixed precision
scaler = GradScaler()
accumulation_steps = 4  # Update parameter setiap 4 batch

for epoch in range(3):
    total_loss = 0
    model.train()

    for i, batch in enumerate(train_loader):
        optimizer.zero_grad()
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        with autocast():  # Mixed precision
            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            loss = outputs.loss / accumulation_steps  # Gradient accumulation

        scaler.scale(loss).backward()  # Mixed precision backward

        if (i + 1) % accumulation_steps == 0:  # Update setiap N batch
            scaler.step(optimizer)
            scaler.update()

        total_loss += loss.item()
        torch.cuda.empty_cache()  # Bersihkan cache GPU untuk menghemat RAM

    print(f"Epoch {epoch + 1}, Loss: {total_loss / len(train_loader)}")


<ipython-input-9-6838da95981e>:7: FutureWarning: `torch.cuda.amp.GradScaler(args...)` is deprecated. Please use `torch.amp.GradScaler('cuda', args...)` instead.
  scaler = GradScaler()
<ipython-input-9-6838da95981e>:20: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with autocast():  # Mixed precision


Epoch 1, Loss: 0.0675213362455368
Epoch 2, Loss: 0.048817460411787035
Epoch 3, Loss: 0.04121967550218105


In [ ]:
# from torch.utils.data import DataLoader
# train_dataset = IMDBDataset(train_df, tokenizer)
# train_loader = DataLoader(train_dataset, batch_size=8, shuffle=True)

# optimizer = AdamW(model.parameters(), lr=2e-5)
# for epoch in range(3):  # 3 epoch cukup untuk fine-tuning
#     model.train()
#     for batch in train_loader:
#         optimizer.zero_grad()
#         input_ids = batch['input_ids'].to(device)
#         attention_mask = batch['attention_mask'].to(device)
#         labels = batch['label'].to(device)
#         outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
#         loss = outputs.loss
#         loss.backward()
#         optimizer.step()

In [ ]:
from sklearn.metrics import accuracy_score
model.eval()
predictions = []
true_labels = []

with torch.no_grad():  # Non-aktifkan perhitungan gradien untuk evaluasi
    for batch in test_loader:
        # Pindahkan batch ke device yang sama dengan model (CPU/GPU)
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['label'].to(device)

        # Dapatkan output model
        outputs = model(input_ids, attention_mask=attention_mask)
        logits = outputs.logits  # Ambil logits dari output

        # Simpan prediksi dan label sebenarnya
        predictions.extend(torch.argmax(logits, dim=1).cpu().numpy())
        true_labels.extend(labels.cpu().numpy())

print("Accuracy:", accuracy_score(true_labels, predictions))

Accuracy: 0.9278


In [ ]:
model.save_pretrained('model/')
tokenizer.save_pretrained('model/')

('model/tokenizer_config.json',
 'model/special_tokens_map.json',
 'model/vocab.txt',
 'model/added_tokens.json')

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

# Simpan ke Drive
model.save_pretrained('/content/drive/MyDrive/model')
tokenizer.save_pretrained('/content/drive/MyDrive/model')

Mounted at /content/drive


('/content/drive/MyDrive/model/tokenizer_config.json',
 '/content/drive/MyDrive/model/special_tokens_map.json',
 '/content/drive/MyDrive/model/vocab.txt',
 '/content/drive/MyDrive/model/added_tokens.json')